In [3]:
from langchain_groq import ChatGroq

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

GROQ_API_KEY =  os.getenv("Groq_API_Key")

llm = ChatGroq(
    temperature=0,
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"
)


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader("https://devjobs.lk/dev-jobs/client/ads/257")
page_data=loader.load().pop().page_content
print(page_data)














DevJobs |             Paid Remote Software Engineer Internship in Sri Lanka | Cadm at Cadmus
                 Apply now for a paid, remote Software Engineer Internship at Cadmus.lk. Join a Sri Lankan digital agency and build real projects using Laravel, React, and APIs.    

























































‚òéÔ∏è 094758888400 (OFFLINE)


üìß support@devjobs.lk

















üí° Knowledge Base


üßë‚Äçüíª Developers



Login


Register

























Vishva Vijenayake
R&D Engineer - Synopsys



                       "DevJobs.lk has truly transformed my job search experience. As an R&D Engineer, I constantly look for roles that challenge me and push the boundaries of innovation. DevJobs.lk delivers exactly that with its intuitive interface and carefully curated opportunities. I was impressed by how quickly I connected with companies that value creativity and technical excellence. I would highly recommend DevJobs.lk to any professiona

In [8]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [9]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer Intern',
 'experience': 'Students or recent graduates',
 'skills': ['HTML',
  'CSS',
  'JavaScript',
  'PHP',
  'Node.js',
  'Laravel',
  'React',
  'RESTful APIs'],
 'description': 'Contribute to real client and internal projects from day one, work with technologies like JavaScript, Laravel, React, Node.js, and APIs, build responsive web apps, backend systems, and custom digital tools.'}

In [10]:
type(json_res)

dict

In [12]:
import pandas as pd

df=pd.read_csv("/content/my_portfolio.csv")
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [15]:
import uuid
import chromadb

client= chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='my_portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row["Techstack"],
            metadatas={"links": row["Links"]},
            ids=[str(uuid.uuid4())]
        )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 31.6MiB/s]


In [17]:
links = collection.query(query_texts=json_res['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [19]:
job = json_res
job['skills']

['HTML',
 'CSS',
 'JavaScript',
 'PHP',
 'Node.js',
 'Laravel',
 'React',
 'RESTful APIs']

In [20]:
job

{'role': 'Software Engineer Intern',
 'experience': 'Students or recent graduates',
 'skills': ['HTML',
  'CSS',
  'JavaScript',
  'PHP',
  'Node.js',
  'Laravel',
  'React',
  'RESTful APIs'],
 'description': 'Contribute to real client and internal projects from day one, work with technologies like JavaScript, Laravel, React, Node.js, and APIs, build responsive web apps, backend systems, and custom digital tools.'}

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Piyumi, a business development executive at InnovD. InnovD is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Piyumi, BDE at Innov.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineer Interns for Your Projects at InnovD

Dear Hiring Manager,

I came across the job description for a Software Engineer Intern at your esteemed organization and was impressed by the exciting opportunities you're offering to students and recent graduates. As a Business Development Executive at InnovD, I'd like to introduce you to our team of skilled engineers who can contribute to your real client and internal projects from day one.

Our engineers are well-versed in technologies like JavaScript, Laravel, React, Node.js, and APIs, and have a proven track record of building responsive web apps, backend systems, and custom digital tools. We believe our team would be a great fit for your projects, and we're excited about the possibility of collaborating with you.

To give you a glimpse into our capabilities, I'd like to share some of our notable projects:

* https://example.com/react-portfolio
* https://example.com/full-stack-js-portfolio
* https://example.com

In [2]:
%pip install -U --quiet langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 4.1 MB/s eta 0:00:00


In [6]:
%pip install -U --quiet langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [14]:
%pip install -U --quiet chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 107.1 MB/s eta 0: